In [ ]:
import pandas as pd
import json
import requests
import petpy

from datetime import datetime, date
from petpy import Petfinder

key = "YOUR KEY HERE"
secret = "YOUR KEY HERE"

datafile_16 = "Resources/FY_2016_Dallas_Animal_Shelter_Data.csv"
datafile_17 = "Resources/FY_2017_Dallas_Animal_Shelter_Data.csv"
datafile_18 = "Resources/FY_2018_Dallas_Animal_Shelter_Data.csv"
datafile_19 = "Resources/FY_2019_Dallas_Animal_Shelter_Data.csv"
datafile_20 = "Resources/FY2020_Dallas_Animal_Shelter_Data.csv"
datafile_21 = "Resources/FY2021_Dallas_Animal_Shelter_Data.csv"

df_2016 = pd.read_csv(datafile_16)
df_2017 = pd.read_csv(datafile_17)
df_2018 = pd.read_csv(datafile_18)
df_2019 = pd.read_csv(datafile_19)
df_2020 = pd.read_csv(datafile_20)
df_2021 = pd.read_csv(datafile_21)

In [ ]:
# MERGE ALL DALLAS FISCAL YEAR DATA INTO ONE DATAFRAME
frames = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021]
dallas_shelter_df = pd.concat(frames)
dallas_shelter_df

In [ ]:
# GET RID OF DUPLICATES (SAME ANIMAL ID AND MONTH ADMITTED TO SHELTER)
dup_animals = dallas_shelter_df.loc[dallas_shelter_df.duplicated(subset = ['Animal Id', 'Month']), 'Animal Id'].unique()
dup_animal_id = pd.DataFrame(dup_animals)
dup_animal_id

In [ ]:
dallas_shelter_clean = dallas_shelter_df[dallas_shelter_df['Animal Id'].isin(dup_animals)==False]
dallas_shelter_clean

In [ ]:
# GET RID OF UNNECESSARY COLUMNS AND GET RID OF ERRORS
dallas_shelter_data_pre = dallas_shelter_clean.dropna(how='all', subset=['Animal Breed'])
dallas_shelter_data = dallas_shelter_data_pre.drop(columns=['Source Id', 'Activity Sequence', 'Receipt Number', 'Census Tract', 'Intake Time', 'Outcome Time','Tag Type', 'Activity Number', 'Service Request Number', 'Unnamed: 16','Unnamed: 27'])
dallas_shelter_data

In [ ]:
# CHECK FORMATTING OF COLUMNS (DTYPES)
dallas_shelter_data['Intake Date'] = pd.to_datetime(dallas_shelter_data['Intake Date'])
dallas_shelter_data['Outcome Date'] = pd.to_datetime(dallas_shelter_data['Outcome Date'])
dallas_shelter_data.info()

In [ ]:
# WRITE CLEAN MERGED DALLAS DATA TO CSV
dallas_shelter_data.to_csv("Resources/DallasData.csv")

In [ ]:
# Make a call to the Petfinder API
pf = Petfinder(key=key, secret=secret)

try:
    # Calls API and puts info in a dictionary
    animals = pf.animals()  

    # Extracting data on specific animals with animal_ids
    animal_ids = []
    for i in animals['animals'][0:3]:
        animal_ids.append(i['id'])
    
    animal_data = pf.animals(animal_id=animal_ids)

    # Returning a pandas DataFrame of the first 10,000 animal results
    animals = pf.animals(results_per_page=100, pages=100, return_df=True) # Max pages is 100
                                                                          # Max results per page is 100
except:
    pass

# Get rid of columns we don't need
animals = animals[['id', 'organization_id', 'type','species', 'age', 'gender', 'size',
                    'coat', 'tags', 'description', 'organization_animal_id', 'status',
                    'status_changed_at', 'published_at', 'breeds.primary', 'breeds.secondary', 
                    'breeds.mixed', 'breeds.unknown', 'colors.primary', 'colors.secondary', 
                    'colors.tertiary', 'contact.address.city', 'contact.address.state', 
                    'contact.address.country', 'animal_id', 
                    'animal_type', 'organization_id']]

# Creates csv file from DataFrame
animals.to_csv("Resources/Animals.csv")